<a href="https://colab.research.google.com/github/Agnessy-ai/deep-learning-image-recognition-3808126/blob/main/Copy_of_Footsteps_Game_Human_vs_Computer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random

# Define the board positions
# The board has 7 circles in a line.
# We can represent positions as integers from -3 to +3.
# -3, -2, -1: Player B's territory
# 0: Center circle (neutral)
# 1, 2, 3: Player A's territory
BOARD_POSITIONS = list(range(-3, 4))
CENTER_POSITION = 0
PLAYER_A_END = 3 # The goal for Player A is to reach position 3
PLAYER_B_END = -3 # The goal for Player B is to reach position -3
STARTING_POINTS = 50 # Each player starts with 50 points
MIN_BID = 1 # The minimum number of points a player must bid each turn

class GameState:
    """Represents the current state of the Footsteps game."""
    def __init__(self, token_position=CENTER_POSITION, player_a_points=STARTING_POINTS, player_b_points=STARTING_POINTS):
        # Initialize the token position to the center
        self.token_position = token_position
        # Initialize player points
        self.player_a_points = player_a_points
        self.player_b_points = player_b_points
        # Flag to indicate if the game is over
        self.is_game_over = False
        # Variable to store the winner or game outcome ('A', 'B', 'Draw', 'Half-Victory A', 'Half-Victory B')
        self.winner = None

    def is_terminal(self):
        """Checks if the current state is a terminal state (game over)."""
        return self.is_game_over

    def get_winner(self):
        """Returns the winner or game outcome if the game is over."""
        return self.winner

    def __str__(self):
        """Provides a user-friendly string representation of the game state."""
        # Create a visual representation of the board
        board_display = ["."] * 7 # Use '.' for empty circles
        # Place 'T' for the token at the correct index (adjusting from -3..3 to 0..6)
        board_display[self.token_position + 3] = "T"
        board_str = "".join(board_display)
        # Include points and game status in the string
        return f"Board: [{board_str}] | A Pts: {self.player_a_points} | B Pts: {self.player_b_points} | Over: {self.is_game_over} | Winner: {self.winner}"

class FootstepsGame:
    """Handles the game logic and state transitions."""
    def __init__(self):
        # Initialize the game with a starting state
        self.state = GameState()
        # Track which player has successive plays if one runs out of points
        self.successive_plays_player = None # 'A', 'B', or None

    def get_current_state(self):
        """Returns the current state of the game."""
        return self.state

    def is_game_over(self):
        """Checks if the game has ended."""
        return self.state.is_terminal()

    def get_outcome(self):
        """Returns the final outcome of the game."""
        return self.state.get_winner()

    def apply_turn(self, bid_a, bid_b):
        """
        Applies a single turn's actions (bids) to the game state.
        Assumes bids are valid (>= MIN_BID and <= remaining points).
        Handles token movement, point deduction, and checks for end conditions.
        Returns the new GameState.
        """
        # If the game is already over, no more turns can be applied
        if self.state.is_terminal():
            return self.state

        current_pos = self.state.token_position
        new_pos = current_pos # Assume no movement initially

        # Determine token movement based on the comparison of bids
        if bid_a > bid_b:
            new_pos = current_pos + 1 # Move one space towards Player B's territory
        elif bid_b > bid_a:
            new_pos = current_pos - 1 # Move one space towards Player A's territory
        # If bid_a == bid_b, new_pos remains the same

        # Deduct the points spent by each player
        new_points_a = self.state.player_a_points - bid_a
        new_points_b = self.state.player_b_points - bid_b

        # Create a new GameState object to represent the state after the turn
        new_state = GameState(new_pos, new_points_a, new_points_b)

        # --- Check for Win Conditions ---
        # A player wins if the token lands on their opponent's end circle.
        if new_state.token_position == PLAYER_A_END:
            new_state.is_game_over = True
            new_state.winner = 'A'
        elif new_state.token_position == PLAYER_B_END:
            new_state.is_game_over = True
            new_state.winner = 'B'
        else:
            # --- Check for Resource Depletion End Conditions (if no win yet) ---
            # These conditions apply if one or both players run out of points.
            if new_points_a == 0 or new_points_b == 0:
                if new_points_a == 0 and new_points_b == 0:
                    # Both players ran out of points simultaneously
                    new_state.is_game_over = True
                    if new_state.token_position == CENTER_POSITION:
                        new_state.winner = 'Draw' # Token in center = Draw
                    elif new_state.token_position > CENTER_POSITION: # Token in A's territory
                         new_state.winner = 'Half-Victory B' # B gets half-victory
                    else: # Token in B's territory
                         new_state.winner = 'Half-Victory A' # A gets half-victory
                elif new_points_a == 0 and new_state.player_b_points > 0:
                    # Player A ran out, Player B has points remaining.
                    # Player B gets successive plays. The game is NOT over yet.
                    # The main game loop needs to handle this successive play logic.
                    pass # Game continues with successive plays
                elif new_points_b == 0 and new_state.player_a_points > 0:
                    # Player B ran out, Player A has points remaining.
                    # Player A gets successive plays. The game is NOT over yet.
                    pass # Game continues with successive plays


        # Update the game's state to the newly calculated state
        self.state = new_state
        return self.state

    def get_legal_bids(self, player):
        """
        Returns a list of legal bids for the specified player in the current state.
        Legal bids are integers from MIN_BID up to the player's remaining points.
        """
        if player == 'A':
            max_bid = self.state.player_a_points
        elif player == 'B':
            max_bid = self.state.player_b_points
        else:
            # Raise an error for an invalid player identifier
            raise ValueError("Invalid player. Use 'A' or 'B'.")

        # Return the range of possible bids
        # Ensure max_bid + 1 is used because range is exclusive of the stop value
        # If max_bid is less than MIN_BID, this range will be empty, indicating no legal bids (points exhausted)
        return list(range(MIN_BID, max_bid + 1))

# --- Agent Classes ---
# Base class for agents and specific implementations (Human, Random, ML)

class Agent:
    """
    Base class for any agent (human or computer) playing Footsteps.
    Defines the interface for choosing a bid.
    """
    def __init__(self, player_id):
        self.player_id = player_id # 'A' or 'B'

    def choose_bid(self, game_state: GameState):
        """
        Abstract method: Agent chooses a bid based on the current game state.
        Must return a valid integer bid.
        """
        raise NotImplementedError("Subclass must implement abstract method 'choose_bid'")

# Human Agent: Takes input from the user
class HumanAgent(Agent):
    def choose_bid(self, game_state: GameState):
        """
        Prompts the human player for a bid and validates the input.
        """
        legal_bids = FootstepsGame().get_legal_bids(self.player_id)
        if not legal_bids:
             print(f"Player {self.player_id} has no points left to bid.")
             return 0 # Should not happen if game end is handled, but as fallback

        while True:
            try:
                # Display available points and legal bid range
                if self.player_id == 'A':
                    points = game_state.player_a_points
                else:
                    points = game_state.player_b_points

                bid_input = input(f"Player {self.player_id}, your points: {points}. Enter your bid ({MIN_BID}-{points}): ")
                bid = int(bid_input) # Convert input to integer

                # Validate the bid
                if bid >= MIN_BID and bid <= points:
                    return bid # Return valid bid
                else:
                    print(f"Invalid bid. Please enter a number between {MIN_BID} and {points}.")
            except ValueError:
                # Handle non-integer input
                print("Invalid input. Please enter a whole number.")

# Random Agent: Computer player that chooses bids randomly
class RandomAgent(Agent):
    def choose_bid(self, game_state: GameState):
        """
        Randomly chooses a legal bid from the available options.
        """
        legal_bids = FootstepsGame().get_legal_bids(self.player_id)
        if not legal_bids:
             return 0 # Should not happen if game end is handled
        return random.choice(legal_bids)

# MLAgent Base Class (for potential future ML computer players)
# This is kept separate from the main Agent base class to distinguish
# learning agents from simple rule-based or human agents.
class MLAgent(Agent):
     def learn(self, old_state: GameState, bid, opponent_bid, new_state: GameState, reward):
        """
        Abstract method: Agent learns from the outcome of a turn.
        """
        pass # Default: no learning

# --- Game Simulation Loop for Human vs Computer ---

def simulate_human_vs_computer(human_player_id='A', computer_agent: Agent = None, max_turns=200):
    """
    Simulates a game of Footsteps between a human player and a computer agent.

    Args:
        human_player_id: The ID ('A' or 'B') for the human player.
        computer_agent: The Agent instance for the computer player.
                        Defaults to a RandomAgent if None.
        max_turns: Maximum number of turns to prevent infinite loops.
    """
    game = FootstepsGame() # Initialize a new game
    turn_count = 0 # Initialize turn counter

    # Set up agents based on who is human and who is computer
    if human_player_id == 'A':
        agent_a = HumanAgent('A')
        agent_b = computer_agent if computer_agent else RandomAgent('B')
        computer_player_id = 'B'
    elif human_player_id == 'B':
        agent_a = computer_agent if computer_agent else RandomAgent('A')
        agent_b = HumanAgent('B')
        computer_player_id = 'A'
    else:
        raise ValueError("human_player_id must be 'A' or 'B'")

    print("--- Footsteps: Human vs Computer ---")
    print(f"You are Player {human_player_id}. Computer is Player {computer_player_id}.")
    print(game.get_current_state()) # Print the initial state

    # Main game loop: continues until the game is over or max turns are reached
    while not game.is_game_over() and turn_count < max_turns:
        turn_count += 1
        print(f"\n--- Turn {turn_count} ---")

        current_state = game.get_current_state() # Get the state at the start of the turn

        # --- Handle Successive Plays ---
        # If one player ran out of points, the other gets successive plays.
        # Only the player with points makes a bid; the other's bid is 0.
        if game.successive_plays_player == 'A':
            print("Player A has successive plays.")
            bid_a = agent_a.choose_bid(current_state)
            bid_b = 0 # Player B cannot bid
            print(f"Player A bids: {bid_a}")
        elif game.successive_plays_player == 'B':
            print("Player B has successive plays.")
            bid_b = agent_b.choose_bid(current_state)
            bid_a = 0 # Player A cannot bid
            print(f"Player B bids: {bid_b}")
        else:
            # Normal turn: Both players make bids. Human gets input, computer uses its logic.
            # Note: Bids are conceptually simultaneous, but collected sequentially here.
            if human_player_id == 'A':
                bid_a = agent_a.choose_bid(current_state) # Human chooses bid
                bid_b = agent_b.choose_bid(current_state) # Computer chooses bid
                print(f"Player A (You) bids: {bid_a}")
                print(f"Player B (Computer) bids: {bid_b}") # Reveal computer bid after human
            else: # Human is Player B
                bid_a = agent_a.choose_bid(current_state) # Computer chooses bid
                bid_b = agent_b.choose_bid(current_state) # Human chooses bid
                print(f"Player A (Computer) bids: {bid_a}")
                print(f"Player B (You) bids: {bid_b}") # Reveal computer bid after human


        # --- Basic Bid Validation ---
        # Check if the chosen bids are legal. This is mainly for the human player input.
        # The RandomAgent should only generate legal bids.
        if bid_a < MIN_BID or bid_a > current_state.player_a_points or \
           bid_b < MIN_BID or bid_b > current_state.player_b_points:
           print(f"Invalid bid detected (A: {bid_a}, B: {bid_b}). Ending game.")
           game.state.is_game_over = True
           game.state.winner = "Invalid Bid" # Indicate game ended due to invalid bid
           break # Exit the game loop

        # --- Apply the Turn ---
        game.apply_turn(bid_a, bid_b)
        new_state = game.get_current_state()

        # Print the state after the turn
        print(new_state)

        # --- Check for Successive Plays Condition ---
        # After points are updated, check if a player has run out of points,
        # triggering successive plays for the opponent in the next turn(s).
        if not new_state.is_terminal():
             if new_state.player_a_points == 0 and new_state.player_b_points > 0:
                 game.successive_plays_player = 'B'
                 # print("Player A ran out of points. Player B has successive plays.") # Printed at start of turn
             elif new_state.player_b_points == 0 and new_state.player_a_points > 0:
                 game.successive_plays_player = 'A'
                 # print("Player B ran out of points. Player A has successive plays.") # Printed at start of turn
             elif new_state.player_a_points == 0 and new_state.player_b_points == 0:
                  # Both ran out - should be caught by is_terminal after apply_turn
                  pass

    # --- Game End ---
    print("\n--- Game Over ---")
    print(f"Outcome: {game.get_outcome()}")
    print(f"Turns Played: {turn_count}")
    if turn_count >= max_turns:
        print("Game ended due to reaching maximum turns.")


# --- Example Usage: Play against the Random Agent ---
if __name__ == "__main__":
    # You can choose to be Player 'A' or 'B'
    # The computer will be the other player, using the RandomAgent logic
    simulate_human_vs_computer(human_player_id='A', computer_agent=RandomAgent('B'))

    # To play as Player B against the computer (Player A):
    # simulate_human_vs_computer(human_player_id='B', computer_agent=RandomAgent('A'))

--- Footsteps: Human vs Computer ---
You are Player A. Computer is Player B.
Board: [...T...] | A Pts: 50 | B Pts: 50 | Over: False | Winner: None

--- Turn 1 ---
Player A, your points: 50. Enter your bid (1-50): 50
Player A (You) bids: 50
Player B (Computer) bids: 12
Board: [....T..] | A Pts: 0 | B Pts: 38 | Over: False | Winner: None

--- Turn 2 ---
Player B has successive plays.
Player B bids: 18
Invalid bid detected (A: 0, B: 18). Ending game.

--- Game Over ---
Outcome: Invalid Bid
Turns Played: 2
